In [ ]:
from pynq import Overlay
from pynq import MMIO
import threading
import time

# Load your custom overlay
motor_control = Overlay("motor_control.bit")

In [ ]:
def set_bit(value, bit):
    return value | (1 << bit)

In [ ]:
# Speed = PWM. Direciton of wheel rotation = GPIO
def control_motors(motor_en, l_forward, r_forward):
    gpio_val = 0

    # Set left motor bits
    gpio_val |= 0x4 if l_forward else 0x8

    # Set right motor bits
    gpio_val |= 0x1 if r_forward else 0x2

    # Write combined value once
    motor_en.write(0x0, gpio_val)

In [ ]:
# PWM SETUP (Using MMIO).....
right_motor = motor_control.axi_timer_0 #PMODB P3 for right PWM
left_motor = motor_control.axi_timer_1  #PMODB P5 for left PWM

# extract register addresses (will be the same for every Axi Timer)
R_TCSR0 = motor_control.ip_dict['axi_timer_0']['registers']['TCSR0']
R_TCSR1 = motor_control.ip_dict['axi_timer_0']['registers']['TCSR1']
R_TCSR0_address = R_TCSR0['address_offset']
R_TCSR1_address = R_TCSR1['address_offset']
R_TCSR0_register = R_TCSR0['fields'] # bit_offset for address
R_TCSR1_register = R_TCSR1['fields']
R_TLR0 = motor_control.ip_dict['axi_timer_0']['registers']['TLR0']
R_TLR1 = motor_control.ip_dict['axi_timer_0']['registers']['TLR1']
R_TLR0_address = R_TLR0['address_offset']
R_TLR1_address = R_TLR1['address_offset']

# create the configuration values for the control register
R_temp_val_0 = 0
R_temp_val_1 = 0

# The PWMA0 bit in TCSR0 and PWMB0 bit in TCSR1 must be set to 1 to enable PWM mode.
R__temp_val_0 = set_bit(R_temp_val_0, R_TCSR0_register['PWMA0']['bit_offset'])
R_temp_val_1 = set_bit(R_temp_val_1, R_TCSR1_register['PWMA1']['bit_offset'])

# The GenerateOut signals must be enabled in the TCSR (bit GENT set to 1). The PWM0
# signal is generated from the GenerateOut signals of Timer 0 and Timer 1, so these
# signals must be enabled in both timer/counters
R_temp_val_0 = set_bit(R_temp_val_0, R_TCSR0_register['GENT0']['bit_offset'])
R_temp_val_1 = set_bit(R_temp_val_1, R_TCSR1_register['GENT1']['bit_offset'])

# The counter can be set to count up or down. UDT
R_temp_val_0 = set_bit(R_temp_val_0, R_TCSR0_register['UDT0']['bit_offset'])
R_temp_val_1 = set_bit(R_temp_val_1, R_TCSR1_register['UDT1']['bit_offset'])

# set Autoreload (ARHT0 = 1)
R_temp_val_0 = set_bit(R_temp_val_0, R_TCSR0_register['ARHT0']['bit_offset'])
R_temp_val_1 = set_bit(R_temp_val_1, R_TCSR1_register['ARHT1']['bit_offset'])

# enable timer (ENT0 = 1)
R_temp_val_0 = set_bit(R_temp_val_0, R_TCSR0_register['ENT0']['bit_offset'])
R_temp_val_1 = set_bit(R_temp_val_1, R_TCSR1_register['ENT1']['bit_offset'])

In [ ]:
L_TCSR0 = motor_control.ip_dict['axi_timer_1']['registers']['TCSR0']
L_TCSR1 = motor_control.ip_dict['axi_timer_1']['registers']['TCSR1']
L_TCSR0_address = L_TCSR0['address_offset']
L_TCSR1_address = L_TCSR1['address_offset']
L_TCSR0_register = L_TCSR0['fields'] # bit_offset for address
L_TCSR1_register = L_TCSR1['fields']
L_TLR0 = motor_control.ip_dict['axi_timer_1']['registers']['TLR0']
L_TLR1 = motor_control.ip_dict['axi_timer_1']['registers']['TLR1']
L_TLR0_address = L_TLR0['address_offset']
L_TLR1_address = L_TLR1['address_offset']

# create the configuration values for the control register
L_temp_val_0 = 0
L_temp_val_1 = 0

# The PWMA0 bit in TCSR0 and PWMB0 bit in TCSR1 must be set to 1 to enable PWM mode.
L_temp_val_0 = set_bit(L_temp_val_0, L_TCSR0_register['PWMA0']['bit_offset'])
L_temp_val_1 = set_bit(L_temp_val_1, L_TCSR1_register['PWMA1']['bit_offset'])

# The GenerateOut signals must be enabled in the TCSR (bit GENT set to 1). The PWM0
# signal is generated from the GenerateOut signals of Timer 0 and Timer 1, so these
# signals must be enabled in both timer/counters
L_temp_val_0 = set_bit(L_temp_val_0, L_TCSR0_register['GENT0']['bit_offset'])
L_temp_val_1 = set_bit(L_temp_val_1, L_TCSR1_register['GENT1']['bit_offset'])

# The counter can be set to count up or down. UDT
L_temp_val_0 = set_bit(L_temp_val_0, L_TCSR0_register['UDT0']['bit_offset'])
L_temp_val_1 = set_bit(L_temp_val_1, L_TCSR1_register['UDT1']['bit_offset'])

# set Autoreload (ARHT0 = 1)
L_temp_val_0 = set_bit(L_temp_val_0, L_TCSR0_register['ARHT0']['bit_offset'])
L_temp_val_1 = set_bit(L_temp_val_1, L_TCSR1_register['ARHT1']['bit_offset'])

# enable timer (ENT0 = 1)
L_temp_val_0 = set_bit(L_temp_val_0, L_TCSR0_register['ENT0']['bit_offset'])
L_temp_val_1 = set_bit(L_temp_val_1, L_TCSR1_register['ENT1']['bit_offset'])

In [ ]:
# GPIO SETUP (Using MMIO)......
l_forward = False;
r_forward = True;

motor_enable_address = motor_control.ip_dict['axi_gpio_0']['phys_addr']
motor_enable = MMIO(motor_enable_address, 8)
motor_enable.write(0x4, 0x0) # Write to tri-state register to configure IO as OUTPUT

_period_ = 10000000 # 50Hz, 20ms
_r_pulse_ = 39
_l_pulse_ = 20

period = int((_period_ & 0x0ffff) *100);
r_pulse = int((_r_pulse_ & 0x07f)*period/100);
l_pulse = int((_l_pulse_ & 0x07f)*period/100);
    
while True:
    control_motors(motor_enable, l_forward, r_forward)

    # Set period and duty cycle
    right_motor.write(R_TLR0_address, period)
    right_motor.write(R_TLR1_address, r_pulse)
    left_motor.write(L_TLR0_address, period)
    left_motor.write(L_TLR1_address, l_pulse)
    
    # Define the LOAD bits (1 << bit_offset)
    R_LOAD0 = 1 << R_TCSR0_register['LOAD0']['bit_offset']
    R_LOAD1 = 1 << R_TCSR1_register['LOAD1']['bit_offset']
    L_LOAD0 = 1 << L_TCSR0_register['LOAD0']['bit_offset']
    L_LOAD1 = 1 << L_TCSR1_register['LOAD1']['bit_offset']

    # Pulse LOAD bits
    right_motor.write(R_TCSR0_address, R_temp_val_0 | R_LOAD0)
    right_motor.write(R_TCSR1_address, R_temp_val_1 | R_LOAD1)
    left_motor.write(L_TCSR0_address, L_temp_val_0 | L_LOAD0)
    left_motor.write(L_TCSR1_address, L_temp_val_1 | L_LOAD1)

    # Re-enable timer (in case LOAD cleared ENT bits)
    right_motor.write(R_TCSR0_address, R_temp_val_0)
    right_motor.write(R_TCSR1_address, R_temp_val_1)
    left_motor.write(L_TCSR0_address, L_temp_val_0)
    left_motor.write(L_TCSR1_address, L_temp_val_1)

    time.sleep(0.5)  # Prevent busy loop (optional)